In [1]:
import requests
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

def request_vision(image_path):
    features = "SmartCrops"
    endpoint = f"https://1507-computervision-we.cognitiveservices.azure.com/computervision/imageanalysis:analyze?"
    api_key = "450e09da0b934eb2bc950eb9f51fa754"

    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }
    params ={
        "api-version": "2024-02-01",
        "features": features,
        "smartcrops-aspect-ratios": "1,1.8,1.4"
    }

    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint, 
                             headers=headers,
                             params=params,
                             data=image_data)

    response_json = response.json()
    print(response_json)

    return response_json

def change_image(image_path):

    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("NanumGothic.ttf", 30)
    except IOError:
        # Arial 폰트가 없으면 기본 폰트를 사용합니다.
        font = ImageFont.load_default()
    
    # request_vision을 통해 데이터를 받아 온다.
    # 받은 데이터를 파싱해서 image에 그린다.

    response_json = request_vision(image_path)
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red", font=font)  

    if "tagsResult" in response_json and "values" in response_json["tagsResult"]:  
        for obj in response_json["tagsResult"]["values"]:  
            print(obj['name'], int(obj['confidence']*100))

    if "smartCropsResult" in response_json and "values" in response_json["smartCropsResult"]:  
        for obj in response_json["smartCropsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            draw.text((x, y - 10), str(obj['aspectRatio']), fill="red", font=font)  


    return image

with gr.Blocks() as demo:

    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

demo.launch()

c:\Users\chch4\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [2]:
import requests
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import io

def request_background_removal(image_path):
    endpoint = 'https://1507-computervision-we.cognitiveservices.azure.com/computervision/imageanalysis:segment?'
    api_key = '450e09da0b934eb2bc950eb9f51fa754'  # 여기에 Azure API 키를 추가하세요.

    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }

    params = {
        'api-version': '2023-02-01-preview',
        'mode': 'backgroundRemoval'
    }

    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint,
                             headers=headers,
                             params=params,  # 'parmas'에서 'params'로 수정
                             data=image_data)
    
    if response.status_code == 200:
        return response.content
    else:
        print(f"Error: {response.status_code}")
        return None

def change_image(image_path):
    image_data = request_background_removal(image_path=image_path)
    if image_data:
        image = Image.open(io.BytesIO(image_data))
        return image
    else:
        return None

with gr.Blocks() as demo2:

    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

demo2.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


{'modelVersion': '2023-10-01', 'metadata': {'width': 1024, 'height': 1024}, 'smartCropsResult': {'values': [{'aspectRatio': 1.0, 'boundingBox': {'x': 0, 'y': 0, 'w': 924, 'h': 924}}, {'aspectRatio': 1.8, 'boundingBox': {'x': 0, 'y': 48, 'w': 924, 'h': 512}}, {'aspectRatio': 1.4, 'boundingBox': {'x': 0, 'y': 0, 'w': 948, 'h': 676}}]}}
0 0 924 924
0 48 924 512
0 0 948 676
